In [1]:
import simpy
import numpy as np

class JusticeSystem:
    def __init__(self, 
                 env, 
                 num_judges, 
                 avg_magistrates_support_time,
                 avg_magistrates_sentencing_probability,
                 avg_magistrates_dismissed_probability,
                 avg_case_management_support_time,
                 avg_case_management_cracked_probability,
                 avg_case_management_dismissed_probability,
                 avg_case_management_sentencing_probability):
        self.env = env
        self.judges = simpy.Resource(env, num_judges)
        self.avg_magistrates_support_time = avg_magistrates_support_time
        self.avg_magistrates_sentencing_probability = avg_magistrates_sentencing_probability
        self.avg_magistrates_dismissed_probability = avg_magistrates_dismissed_probability
        self.avg_case_management_support_time = avg_case_management_support_time
        self.avg_case_management_cracked_probability = avg_case_management_cracked_probability
        self.avg_case_management_dismissed_probability = avg_case_management_dismissed_probability
        self.avg_case_management_sentencing_probability = avg_case_management_sentencing_probability

        # Initialize counters
        self.magistrates_stack = 0
        self.case_management_stack = 0
        self.customers_processed = 0

        # For simplification, only track customers as Done or Not Done:
        self.magistrates_done_probability = self.avg_magistrates_sentencing_probability + self.avg_magistrates_dismissed_probability
        self.case_management_done_probability = self.avg_case_management_cracked_probability + self.avg_case_management_sentencing_probability

    def support(self):
        self.magistrates_stack += 1
        random_time = max(1, np.random.normal(self.avg_magistrates_support_time, 4))
        yield self.env.timeout(random_time) 
        if np.random.uniform(0, 1) < self.magistrates_done_probability:
            self.customers_processed += 1
            self.magistrates_stack -= 1
        else:
            
                yield self.case_management_support()
                self.magistrates_stack -= 1

    def case_management_support(self):
        with self.judges.request() as request:
                yield request
        self.case_management_stack += 1
        random_time = max(1, np.random.normal(self.avg_case_management_support_time, 4))
        yield self.env.timeout(random_time)
        self.customers_processed += 1
        self.case_management_stack -= 1

    def simulate(self):
        while True:
            self.support()

In [2]:
# Example usage:
env = simpy.Environment()
justice_system = JusticeSystem(env, num_judges=1, avg_magistrates_support_time=5,
                               avg_magistrates_sentencing_probability=0.5,
                               avg_magistrates_dismissed_probability=0.2,
                               avg_case_management_support_time=5,
                               avg_case_management_cracked_probability=0.35,
                               avg_case_management_dismissed_probability=0.15,
                               avg_case_management_sentencing_probability=0.5)

env.process(justice_system.simulate())
env.run(until=10)  # Run the simulation for 10 time units

KeyboardInterrupt: 

In [ ]:
justice_system.customers_processed